In [151]:
import nltk
from tqdm import tqdm
import xlrd
import pandas as pd
import random
from textblob import TextBlob
import numpy as np
import IPython
import re
from nltk.corpus import stopwords
import json
from gensim.models import Word2Vec
import time
from nltk import sent_tokenize
from senticnet.senticnet import SenticNet
from wordcloud import WordCloud
# import matplotlib.pyplot as plt


In [90]:
adj = ['JJ','JJR','JJS']
adv = ['RB','RBR','RBS']
vb = ['VB','VBD','VBG','VBN','VBP','VBZ']
nn = ['NN','NNS']

sn = SenticNet()

def review_to_words(review_text): 
    if '(Reuters) -' in review_text:
        review_text = review_text.split('(Reuters) -')[1]
    if '*' in review_text:
        review_text = review_text.split('*')[1]
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    words = letters_only.split()                             
    _stopwords = set(stopwords.words("english"))
    _stopwords = nltk.corpus.stopwords.words('english')
    _stopwords.append('would')
    _stopwords.append('kmh')
    _stopwords.append('mph')
    _stopwords.append('  ')
    _stopwords.append('Reuters')
    _stopwords.append('reuters')
    # _stopwords = []
    meaningful_words = [w for w in words if w not in _stopwords]
    return meaningful_words

# d = enchant.Dict("en_US")
# import spacy
# nlp = spacy.load('en')
# def stem_and_check(word):
#     word = nlp(word)
#     return word[0].lemma_
    # word = inf.singularize(word)
    # word = nltk.PorterStemmer().stem(word)
    # if d.check(word):
    #    return word
    # suggest_words = d.suggest(word)
    # if len(suggest_words) == 0:
    #    return word
    # return suggest_words[0]

def my_read(path):
    file = open(path)
    words = []
    for line in file.readlines():
        words.append(line.strip())
    return words

    
def output_cloud(count,name):
    # 云图
    text = '' 
    for key,value in count.items():
        text += (key+' ') * (value)
    wc = WordCloud(
        width=1000,
        height=600,
        max_font_size=100,      #字体大小
        min_font_size=10,
        collocations=False, 
        max_words=1000
    )
    wc.generate(text)
    wc.to_file(name+'.png') #图片保存



def vote(results,datas):
    if len(datas) != len(results):
        raise("vote error")
    vote_dict = {}
    date_set = set([data['date'] for data in datas])
    company_set = set([data['company'] for data in datas])
    for company in company_set:
        vote_dict[company] = {}
        for date in date_set:
            vote_dict[company][date] = set()
    for idx in range(0,len(datas)):
        date = datas[idx]['date']
        company = datas[idx]['company']         
        vote_dict[company][date].add(idx)
        
    new_results = []
    print('voting')
    for i in range(0,len(results)):
        count = 0
        company = datas[i]['company']
        date = datas[i]['date']
        for idx in vote_dict[company][date]:
            count += results[idx]
        if count < 0:  
            new_results.append(np.float64(-1))
        else:
            new_results.append(np.float64(1))
    return np.array(new_results)

def accuracy(y,y2):
    if len(y) != len(y2):
        raise Exception("error")
    count = 0
    for i in range(0,len(y)):
        if y[i] == y2[i]:
            count += 1
    return count/len(y2)

# concept_info = sn.concept('love')
# polarity_value = sn.polarity_value('love')
# polarity_intense = sn.polarity_intense('love')
# moodtags = sn.moodtags('love')
# semantics = sn.semantics('love')
# sentics = sn.sentics('love') 
 
def sentiment_score(text):
    t = TextBlob(text)
    score = t.sentiment.polarity
    return score
    # tokens = nltk.word_tokenize(text)
    # pos_tags = nltk.pos_tag(tokens)
    # score = 0
    # count = 0
    # for word,tag in pos_tags:
    #   if word in sn.data.keys():
    #    score += float(sn.polarity_intense(word))
    #    count += 1
    #    # print(word,sn.polarity_intense(word))
    # if count == 0: #mid
    #   return -1 
    # return score/count
    
def load_data(path):
    workbook = xlrd.open_workbook(path)
    worksheet = workbook.sheet_by_index(0)
    contents = worksheet.col_values(1)
    companies = worksheet.col_values(2)
    prices = worksheet.col_values(3)
    dates = worksheet.col_values(4)
    datas = []
    for i in tqdm(range(0,len(contents))):
    # if '*' not in contents[i]:
        # if companies[i] != 'Apple Inc.':
        #     continue
        price_list = json.loads(prices[i])   
        if price_list[2] == price_list[1]:
            continue
        data = {}
        data['content'] = contents[i]
        sents = sent_tokenize(data['content'])
        data['tokens'] = []
        data['tags'] = []
        for sent in sents:
            token = review_to_words(sent) # 去停用词会影响词性标注吗？？
            data['tokens'].append(token)
            data['tags'].append(nltk.pos_tag(token))
        data['company'] = companies[i]
        data['rate'] = (price_list[2]-price_list[1])/price_list[1]
        data['date'] = dates[i]
        datas.append(data)

    return datas

In [222]:
def train_sent_dict(datas):
    count = {}

    POS = 0
    NEG = 0

    pos_count = 0
    neg_count = 0

    N = 0 #len of tokens
        
    for data in datas:
        for tokens in data['tokens']: 
            N += len(tokens)
            rate = data['rate'] # 选当天的股票变化判断涨跌，因为相关度当天的最高
            if rate>0:
                pos_count += 1
                POS += len(tokens)
                for token in tokens:
                    if len(token) < 3:
                        continue
                    if 'not' in tokens:
                        token = 'not_'+token
                    if token in count.keys():
                        count[token]['pos'] += 1
                        count[token]['pos_rate'] += rate
                    else:
                        count[token] = {'pos':1,'neg':0,'pos_rate':rate,'neg_rate':0} 

            if rate<0:
                neg_count += 1
                NEG += len(tokens)
                for token in tokens:
                    if len(token) < 3:
                        continue
                    if 'not' in tokens:
                        token = 'not_'+token
                    if token in count.keys():
                        count[token]['neg'] += 1
                        count[token]['neg_rate'] -= rate
                    else:
                        count[token] = {'pos':0,'neg':1,'pos_rate':0,'neg_rate':-rate}
                        ## freq
    copy = count.copy()
    sent_words = [] # PD>0.3情感值

    freq_pos = pos_count/len(datas) 
    freq_neg = neg_count/len(datas)

    # DS sent and PMI sent
    for word,value in copy.items():
        if value['pos']+value['neg']<len(datas)/100:
            del count[word]
            continue
        freq_w_pos = value['pos']/len(datas)
        freq_w_neg = value['neg']/len(datas)
        freq_w = (value['pos']+value['neg'])/len(datas)
        if freq_w_pos*N == 0:
            PMI_w_pos = 0
        else:
            PMI_w_pos = np.log2(freq_w_pos*N/freq_w*freq_pos)
        if freq_w_neg*N == 0:
            PMI_w_neg = 0
        else:
            PMI_w_neg = np.log2(freq_w_neg*N/freq_w*freq_neg)
        count[word]['PMI_sent'] = PMI_w_pos - PMI_w_neg

        pos = value['pos']/len(datas)
        neg = value['neg']/len(datas)
        value['PD'] = (pos-neg)/(pos+neg) # polarity difference
        if abs(value['PD']) > 0.3 and nltk.pos_tag([word])[0][1] in adj+adv:  
            sent_words.append(word)
        count[word]['sent'] = value['PD']*value['PD'] * np.sign(value['PD'])

        pos_rate = value['pos_rate']/len(datas)
        neg_rate = value['neg_rate']/len(datas)
        value['PD_rate'] = (pos_rate-neg_rate)/(pos_rate+neg_rate) # polarity difference
        count[word]['sent_rate'] = value['PD_rate']*value['PD_rate'] * np.sign(value['PD_rate'])
    return count


In [343]:
def news2vector(datas,count,bl_sent):
    for idx in range(0,len(datas)):    
        datas[idx]['DsVector'] = [0,0,0,0]
        datas[idx]['DsVector_rate'] = [0,0,0,0]
        datas[idx]['SnVector'] = [0,0,0,0]
        datas[idx]['BlVector'] = [0,0,0,0]
        datas[idx]['PmiVector'] = [0,0,0,0]
        datas[idx]['ContextVector'] = [0,0,0,0]
        
        
        for tags in datas[idx]['tags']:
            for word,tag in tags:
                if tag in adj:
                    if word in count.keys():
                        datas[idx]['DsVector'][0] += count[word]['sent']
                        datas[idx]['DsVector_rate'][0] += count[word]['sent_rate']
                        datas[idx]['PmiVector'][0] += count[word]['PMI_sent']
                    if word in sn.data.keys():
                        datas[idx]['SnVector'][0] += float(sn.polarity_intense(word))
                    if word in bl_sent.keys():
                        datas[idx]['BlVector'][0] += bl_sent[word]
                elif tag in adv:
                    if word in count.keys():
                        datas[idx]['DsVector'][1] += count[word]['sent']
                        datas[idx]['DsVector_rate'][1] += count[word]['sent_rate']
                        datas[idx]['PmiVector'][1] += count[word]['PMI_sent']
                    if word in sn.data.keys():
                        datas[idx]['SnVector'][1] += float(sn.polarity_intense(word))
                    if word in bl_sent.keys():
                        datas[idx]['BlVector'][1] += bl_sent[word]  
                elif tag in nn:
                    if word in count.keys():
                        datas[idx]['DsVector'][2] = count[word]['sent']
                        datas[idx]['DsVector_rate'][2] += count[word]['sent_rate']
                        datas[idx]['PmiVector'][2] += count[word]['PMI_sent']
                    if word in sn.data.keys():
                        datas[idx]['SnVector'][2] += float(sn.polarity_intense(word))
                    if word in bl_sent.keys():
                        datas[idx]['BlVector'][2] += bl_sent[word]
                elif tag in vb:
                    if word in count.keys():
                        datas[idx]['DsVector'][3] = count[word]['sent']
                        datas[idx]['DsVector_rate'][3] += count[word]['sent_rate']
                        datas[idx]['PmiVector'][3] += count[word]['PMI_sent']
                    if word in sn.data.keys():
                        datas[idx]['SnVector'][3] += float(sn.polarity_intense(word))
                    if word in bl_sent.keys():
                        datas[idx]['BlVector'][3] += bl_sent[word]


In [344]:
def news2vector2(datas,count,bl_sent):
    for idx in range(0,len(datas)):
        datas[idx]['DsVector_rate'] = [0,0,0,0]
        for tags in datas[idx]['tags']:
            for word,tag in tags:
                if tag in adj:
                    if word in count.keys():
                        datas[idx]['DsVector_rate'][0] += count[word]['sent_rate']

                elif tag in adv:
                    if word in count.keys():
                        datas[idx]['DsVector_rate'][1] += count[word]['sent_rate']

                elif tag in nn:
                    if word in count.keys():
                        datas[idx]['DsVector_rate'][2] += count[word]['sent_rate']

                elif tag in vb:
                    if word in count.keys():
                        datas[idx]['DsVector_rate'][3] += count[word]['sent_rate']
        # datas[idx]['DsVector'] = [adv_score,adv_score,noun_score,verb_score]


In [229]:
import pickle
datas = pickle.load(open('/home/stocksentiment/datas.pkl','rb'))
count = pickle.load(open('/home/stocksentiment/sent_dict.pkl', 'rb'))
print('load finish')

load finish


In [101]:
path = r'~/Dynamic-Financial-News-Collection-and-Analysis/labeled_data.xls'

# datas = load_data(path)
# count = train_sent_dict(datas)

# import pickle
# output = open('sent_dict.pkl', 'wb')
# input = open('sent_dict.pkl', 'rb')
# s = pickle.dump(count, output)
# output.close()
# clf2 = pickle.load(input)
# input.close()
# print clf2.predict(X[0:1])

In [102]:
## 新闻情感值和股价的相关度
workbook = xlrd.open_workbook(path)
worksheet = workbook.sheet_by_index(0)
contents = worksheet.col_values(1)
companies = worksheet.col_values(2)
prices = worksheet.col_values(3)
dates = worksheet.col_values(4)

In [116]:
rates = []
score_list = []

for i in range(0,len(contents)):
    rate = []
    price_list = json.loads(prices[i])
    for idx in range(0,6):
        rate.append((price_list[idx+1]-price_list[idx])/price_list[idx])
    rates.append(rate)
    score_list.append(sentiment_score(contents[i]))
    
## 合并一天新闻
# for i in tqdm(range(0,len(contents))):
#     rate = []
#     price_list = json.loads(prices[i])
#     for idx in range(0,6):
#       rate.append((price_list[idx+1]-price_list[idx])/price_list[idx])
#     if rate in rates:
#         idx = rates.index(rate)
#         score_list[idx] += sentiment_score(contents[i])
#         continue
#     rates.append(rate)
#     score_list.append(sentiment_score(contents[i]))

# 情感极性与六天内（包括）新闻涨跌比率的相关度


In [115]:
# 股票与新闻情感相关性
fiveday_rate_list = []
for i in range(0,6):
    rate = [x[i] for x in rates]
    data = {
        'scores':score_list,
        'rates':rate
        }

    df = pd.DataFrame(data)
   # print(df)
    print(df.corr("kendall"))

           rates    scores
rates   1.000000  0.033801
scores  0.033801  1.000000
           rates    scores
rates   1.000000  0.002254
scores  0.002254  1.000000
           rates    scores
rates   1.000000  0.001984
scores  0.001984  1.000000
           rates    scores
rates   1.000000 -0.000839
scores -0.000839  1.000000
           rates    scores
rates   1.000000 -0.000409
scores -0.000409  1.000000
           rates    scores
rates   1.000000 -0.002007
scores -0.002007  1.000000


In [152]:
# wordvec
tokens = []
for data in datas:
    for ts in data['tokens']:
        tokens.append(ts)
model = Word2Vec(sentences = tokens,min_count = 10)

KeyboardInterrupt: 

In [121]:
from nltk.tokenize import WordPunctTokenizer  
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection
from sklearn.naive_bayes import GaussianNB
import xlrd,xlwt
import numpy as np
# import spacy
from gensim.models import Word2Vec
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import to_categorical

sentences = [data['content'] for data in datas]

bag_of_keywords = set(['fail', 'success', 'win', 'drop', 'rise', 'shrink', 'jump', 'gain', 'down', 'up'])
stop = False
bok_size = 100
for i in range(10):
	new_words = []
	if stop:break
	for k in bag_of_keywords:
		if k in model.wv.vocab.keys():# wv = wordvector
			new_words.extend(model.most_similar(k))
for n in new_words:
	if n[0].islower() and len(n[0])>3 and n[0].isalpha():
		bag_of_keywords.add(n[0])
		if len(bag_of_keywords) == bok_size:
			stop = True
			break

'''fit():计算数据的参数，\mu（均值），\sigma（标准差），并存储在对象中（例如实例化的CountVectorizer()等）。
transform():将这些参数应用到数据集，进行标准化（尺度化）。'''

## Bag of keywords 统计词语的两个api
bag_of_keywords = np.array(list(bag_of_keywords))
bok_tfidf = TfidfVectorizer(lowercase = False, min_df = 1, vocabulary=bag_of_keywords)
X_bok_tfidf = bok_tfidf.fit_transform(sentences)
X_bok_tfidf = X_bok_tfidf.toarray()

## Category tag
category_tags = set(['published','presented','unveil','investment','bankrupt','acquisition','government'
                     'sue','lawsuit','highlights'])
stop = False
cate_size = 100

for _ in range(10):
    new_words = []
    if stop:break
    for k in category_tags:
        if k in model.wv.vocab.keys():
            new_words.extend(model.most_similar(k))
    for n in new_words:
        if n[0].islower() and len(n[0])>3 and n[0].isalpha():
            category_tags.add(n[0])
            if len(category_tags) == cate_size:
                stop = True
                break


category_tags = np.array(list(category_tags))

ct_tfidf = TfidfVectorizer(lowercase = False, min_df = 1, vocabulary = category_tags)
X_ct_idf = ct_tfidf.fit_transform(sentences)
X_ct_idf = X_ct_idf.toarray()

full_tfidf = TfidfVectorizer(lowercase=False, min_df = 1,vocabulary=bag_of_keywords,use_idf=False)
X_full_tfidf = full_tfidf.fit_transform(sentences)
X_full_tfidf = X_full_tfidf.toarray()


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


In [137]:
X_ct_idf[1]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.34411929, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.60995393, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.4684358 ,
       0.        , 0.        , 0.34388431, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [141]:
(X_full_tfidf==X_bok_tfidf).all()

False

In [127]:
Y

[-1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -

In [144]:
# x_bok RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestClassifier
from keras.utils import to_categorical
import matplotlib.pyplot as plt

print('X_bok_tfidf')
X = X_bok_tfidf
train_x,test_x,train_y,test_y=model_selection.train_test_split(X,Y,test_size=0.2,shuffle=False)
clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
# clf = LinearRegression()
clf.fit(np.array(train_x), np.array(train_y))
predict_y = clf.predict(test_x)
print('accuracy：',clf.score(np.array(test_x), np.array(test_y))) 
print('recall：',recall_score(test_y,clf.predict(test_x),average = 'macro'))
print('precision：',precision_score(test_y, clf.predict(test_x), average='macro'))
    
print('X_ct_idf')
X = X_ct_idf
train_x,test_x,train_y,test_y=model_selection.train_test_split(X,Y,test_size=0.2,shuffle=False)
clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
# clf = LinearRegression()
clf.fit(np.array(train_x), np.array(train_y))
predict_y = clf.predict(test_x)
print('accuracy：',clf.score(np.array(test_x), np.array(test_y))) 
print('recall：',recall_score(test_y,clf.predict(test_x),average = 'macro'))
print('accuracy：',precision_score(test_y, clf.predict(test_x), average='macro'))

print('X_full_tfidf')
X = X_full_tfidf
train_x,test_x,train_y,test_y=model_selection.train_test_split(X,Y,test_size=0.2,shuffle=False)
clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
# clf = LinearRegression()
clf.fit(np.array(train_x), np.array(train_y))
predict_y = clf.predict(test_x)
print('accuracy：',clf.score(np.array(test_x), np.array(test_y))) 
print('recall：',recall_score(test_y,clf.predict(test_x),average = 'macro'))
print('accuracy：',precision_score(test_y, clf.predict(test_x), average='macro'))

X_bok_tfidf
accuracy： 0.5206199757735932
recall： 0.4999558721922025
precision： 0.497998753016745
X_ct_idf
accuracy： 0.5209503358660941
recall： 0.5000862740324018
accuracy： 0.52898770329963
X_full_tfidf
accuracy： 0.5209641008699483
recall： 0.5000856567806256
accuracy： 0.5535724715342379


In [145]:
# x_bok RandomForestClassifier
print('X_bok_tfidf')
X = X_bok_tfidf
train_x,test_x,train_y,test_y=model_selection.train_test_split(X,Y,test_size=0.2,shuffle=False)
clf = GaussianNB()
# clf = LinearRegression()
clf.fit(np.array(train_x), np.array(train_y))
predict_y = clf.predict(test_x)
print('accuracy：',clf.score(np.array(test_x), np.array(test_y))) 
print('recall：',recall_score(test_y,clf.predict(test_x),average = 'macro'))
print('precision：',precision_score(test_y, clf.predict(test_x), average='macro'))

vote_predict_y = vote(predict_y,datas[-len(test_x):])
vote_recall = recall_score(test_y,vote_predict_y,average = 'macro')
vote_precision = precision_score(test_y, vote_predict_y, average='macro')

    
print('X_ct_idf')
X = X_ct_idf
train_x,test_x,train_y,test_y=model_selection.train_test_split(X,Y,test_size=0.2,shuffle=False)
clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
# clf = LinearRegression()
clf.fit(np.array(train_x), np.array(train_y))
predict_y = clf.predict(test_x)
print('accuracy：',clf.score(np.array(test_x), np.array(test_y))) 
print('recall：',recall_score(test_y,clf.predict(test_x),average = 'macro'))
print('accuracy：',precision_score(test_y, clf.predict(test_x), average='macro'))

print('')

print('X_full_tfidf')
X = X_full_tfidf
train_x,test_x,train_y,test_y=model_selection.train_test_split(X,Y,test_size=0.2,shuffle=False)
clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
# clf = LinearRegression()
clf.fit(np.array(train_x), np.array(train_y))
predict_y = clf.predict(test_x)
print('accuracy：',clf.score(np.array(test_x), np.array(test_y))) 
print('recall：',recall_score(test_y,clf.predict(test_x),average = 'macro'))
print('accuracy：',precision_score(test_y, clf.predict(test_x), average='macro'))

X_bok_tfidf
accuracy： 0.5072266270234556
recall： 0.49985112388246794
precision： 0.49983023404856497
voting
X_ct_idf
accuracy： 0.5209503358660941
recall： 0.5000862740324018
accuracy： 0.52898770329963

X_full_tfidf
accuracy： 0.5209641008699483
recall： 0.5000856567806256
accuracy： 0.5535724715342379


In [146]:
# neg pos 词
pos_words = {}
neg_words = {}
for word in sent_words:
   if count[word]['sent'] > 0:
      pos_words[word.lower()] = count[word]['pos']+count[word]['neg']
   else:
      neg_words[word.lower()] = count[word]['pos']+count[word]['neg']

output_cloud(pos_words,'pos')
output_cloud(neg_words,'neg')



NameError: name 'sent_words' is not defined

In [147]:
count['company']

{'PD': 0.022119462732212183,
 'PD_rate': 0.04246705264244967,
 'PMI_sent': 0.12259288489754994,
 'neg': 371555,
 'neg_rate': 5250.115713877801,
 'pos': 388364,
 'pos_rate': 5715.806092502332,
 'sent': 0.0004892706315617237,
 'sent_rate': 0.0018034505601365913}

In [153]:
# neg pos 词
pos_words = {}
neg_words = {}
for word in count.keys():
   if count[word]['sent'] > 0:
      pos_words[word.lower()] = count[word]['pos']+count[word]['neg']
   else:
      neg_words[word.lower()] = count[word]['pos']+count[word]['neg']

output_cloud(pos_words,'pos')
output_cloud(neg_words,'neg')

In [156]:
output_cloud(pos_words,'pos')
output_cloud(neg_words,'neg')

In [44]:
## 求于bl词典的覆盖率
bl_sent = {}
bl_pos = my_read('/home/stocksentiment/Dynamic-Financial-News-Collection-and-Analysis/sentiment_analysis/bl/positive.txt')  # 4783
bl_neg = my_read('/home/stocksentiment/Dynamic-Financial-News-Collection-and-Analysis/sentiment_analysis/bl/negative.txt')  # 2006


for word in bl_pos:
    bl_sent[word] = 1
for word in bl_pos:
    bl_sent[word] = -1
# pc = 0
# for word in pos_words:
#    if word in bl_pos:
#       pc += 1
# pos_accuracy = pc/len(pos_words)  # 0.2857142857142857

# nc = 0
# for word in neg_words:
#    if word in bl_neg:
#       nc += 1
# neg_accuracy = nc/len(neg_words)  # 0.18

In [28]:
## context sentiment dict
# sent_words = [word.lower() for word in sent_words]
# feature_words = {}
# sentiment_feature = {}

# for data in tqdm(datas):
#     for tags in data['tags']:
#         for word,tag in tags:
#             if tag not in nn or len(word)<3: # vb+nn
#                 continue
#             # word = stem_and_check(word)
#             if word not in feature_words.keys():
#                 feature_words[word] = 1
#             else:
#                 feature_words[word] += 1

# # avg_f = sum([item[1] for item in feature_words.items()])/len(feature_words.keys())
# res = sorted(feature_words.items(),key=lambda feature_words:feature_words[1],reverse=True)
# words = res[:400]
# # for word,value in tqdm(copy.items()):
# #     if value<avg_f+200:
# #         del feature_words[word]

# feature_words = [inf.singularize(word).lower() for word,freq in words]

# sf_len = 0
# for data in tqdm(datas):
#     rate = data['rate']
#     for tokens in data['tokens']:
#         token_dict = {}
#         for token in tokens:
#             token_dict[inf.singularize(token).lower()] = token
#         _tokens = [inf.singularize(token).lower() for token in tokens]

#         for w in list(set(sent_words).intersection(set(_tokens))):
#             for f in list(set(feature_words).intersection(set(_tokens))):
#                 if f != w:
#                     if abs(_tokens.index(w)-_tokens.index(f))<3 and ',' not in data['content'][min(data['content'].index(token_dict[f]),data['content'].index(token_dict[w])):max(data['content'].index(token_dict[f]),data['content'].index(token_dict[w]))]:
#                         sf_len += 1
#                         if f not in sentiment_feature.keys():
#                             sentiment_feature[f] = {}
#                             if rate > 0:
#                                 sentiment_feature[f][w] = {'pos':1,'neg':0}
#                             if rate < 0:
#                                 sentiment_feature[f][w] = {'pos':0,'neg':1}
#                         else:
#                             if w not in sentiment_feature[f].keys():
#                                 sentiment_feature[f][w] = {'pos':0,'neg':0}
#                             if rate > 0:
#                                 sentiment_feature[f][w]['pos'] += 1
#                             if rate < 0:
#                                 sentiment_feature[f][w]['neg'] += 1

# avg_sf = sf_len/len(sentiment_feature.keys())
# copy = sentiment_feature.copy()

# for f,v in tqdm(sentiment_feature.items()):
#     for w,value in v.items():
#         if value['pos']+value['neg']<avg_sf: #avg_sf
#             # print(f,w,value)
#             # del sentiment_feature[f][w]
#             sentiment_feature[f][w]['sent'] = 0
#             continue
#         pos = value['pos']/POS
#         neg = value['neg']/NEG
        
#         value['PD'] = (pos-neg)/(pos+neg) # polarity difference
#         sentiment_feature[f][w]['sent'] = value['PD'] * value['PD'] * np.sign(value['PD'])

# res = sorted(sentiment_feature.items(),key=lambda sentiment_feature:sentiment_feature[1]['sent'],reverse=False)

## company word
# company_pos = {}
# company_neg = {}
# for key,value in sentiment_feature.items():
#    if 'company' in key:
#       if sentiment_feature[key]['sent'] > 0:
#          company_pos[key.split('_')[0]] = sentiment_feature[key]['pos']
#       else:
#          company_neg[key.split('_')[0]] = sentiment_feature[key]['neg']

# output_cloud(company_pos,'company_pos')
# output_cloud(company_neg,'company_neg')


# for r in res[:30]:
#    print(r[0],r[1]['sent'],r[1]['pos']+r[1]['neg'])
#    print(' ')

# print(' ')
# print('========================')

# for r in res[-40:]:
#    print(r[0],r[1]['sent'],r[1]['pos'],r[1]['neg'])
#    print(' ')

# 展示
# pos_res = {}
# for r in res:
#     if r[1]['sent'] == 1.0:
#         pos_res[r[0]] = r[1]['pos']+r[1]['neg']
# pos_res = sorted(pos_res.items(),key=lambda pos_res:pos_res[1],reverse=True)
# for r in pos_res[:20]:
#     print(r[0],'1.0',r[1])
#     print(' ')

# print(' ')
# print('========================')
# print(' ')

# neg_res = {}
# for r in res:
#     if r[1]['sent'] == -1.0:
#         neg_res[r[0]] = r[1]['pos']+r[1]['neg']
# neg_res = sorted(neg_res.items(),key=lambda neg_res:neg_res[1],reverse=True)
# for r in neg_res[:20]:
#     print(r[0],'-1.0',r[1])
#     print(' ')
# for sf,value in sentiment_feature.items():
#    if value['pos']+value['neg'] > avg_sf:
#       print(sf,freq)

In [33]:
news2vector(datas,count,bl_sent)

100%|██████████| 363237/363237 [1:15:57<00:00, 27.32it/s] 


In [355]:
output = open('datas.pkl', 'wb')
pickle.dump(datas, output)
print('dump finish')

dump finish


{'BlVector': [0, 0, -4, 0],
 'ContextVector': [0, 0, 0, 0],
 'DsVector': [0.009382414055471773,
  0.003466796193975887,
  0.00011491759888645986,
  0.0008279551668078367],
 'DsVector_rate': [0.04473495161500582,
  0.01280244453155389,
  0.2797177700726252,
  0.10969579071672703],
 'PmiVector': [2.3897680592562054,
  0.9182778690101259,
  12.363459195184863,
  4.756092484475477],
 'SnVector': [-0.6450000000000007,
  -2.5869999999999997,
  12.297000000000002,
  1.055],
 'company': 'Ameriprise Financial',
 'content': 'NEW YORK  (Reuters) - H&R Block Inc will pay as much as $20.2 million to settle a New York lawsuit accusing it of fraudulently marketing retirement accounts that caused hundreds of thousands of mostly lower-income clients to lose money.    New York Attorney General Andrew Cuomo said the accord calls for the largest U.S. tax preparer to refund $11.4 million to $19.4 million of fees to customers nationwide who opened one of its Express IRAs, a type of individual retirement acc

In [88]:
vote_dict = {}
date_set = set([data['date'] for data in datas])
company_set = set([data['company'] for data in datas])
for company in company_set:
    vote_dict[company] = {}
    for date in date_set:
        vote_dict[company][date] = set()
for idx in range(0,len(datas)):
    date = datas[idx]['date']
    company = datas[idx]['company']         
    vote_dict[company][date].add(idx)

In [87]:
vote_dict['Harley-Davidson']

{'2017-10-13': set(),
 '2016-07-08': {302925, 302947, 302948, 303008},
 '2017-07-20': {338115, 338196, 338207},
 '2014-05-02': set(),
 '2013-09-12': set(),
 '2014-10-24': set(),
 '2016-05-31': set(),
 '2015-03-16': set(),
 '2012-05-08': set(),
 '2012-10-22': set(),
 '2017-11-24': set(),
 '2014-08-15': set(),
 '2017-01-05': set(),
 '2015-06-16': set(),
 '2014-04-10': set(),
 '2012-10-25': {146601},
 '2017-11-09': set(),
 '2016-11-15': set(),
 '2015-07-17': set(),
 '2011-09-13': set(),
 '2015-01-06': set(),
 '2015-04-15': set(),
 '2017-01-23': set(),
 '2010-01-07': set(),
 '2010-12-28': set(),
 '2017-06-28': set(),
 '2017-11-01': set(),
 '2012-02-09': set(),
 '2010-11-01': set(),
 '2014-02-18': set(),
 '2011-09-19': set(),
 '2015-10-08': set(),
 '2012-06-06': set(),
 '2013-10-02': set(),
 '2011-03-15': set(),
 '2012-06-20': set(),
 '2013-11-15': set(),
 '2012-05-23': set(),
 '2012-09-12': set(),
 '2013-03-06': set(),
 '2013-08-15': set(),
 '2015-09-10': set(),
 '2018-03-12': set(),
 '201

In [93]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestClassifier
from keras.utils import to_categorical
import matplotlib.pyplot as plt
# RandomForest
X1 = [data['DsVector'] for data in datas]
X2 = [data['SnVector'] for data in datas]
X3 = [data['BlVector'] for data in datas]
X4 = [data['PmiVector'] for data in datas]
X5 = [data['DsVector_rate'] for data in datas]

Xs = {'DsVector':X1,'SnVector':X2,'BlVector':X3,'PmiVector':X4,'DsVector_rate':X5}

Y = [np.sign(data['rate']) for data in datas]
for vectorname in Xs.keys():
    print(vectorname+'==============')
    X = Xs[vectorname]
    train_x,test_x,train_y,test_y = model_selection.train_test_split(X,Y,test_size=0.2,shuffle=False)

    clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)

    clf.fit(np.array(train_x), np.array(train_y))

    predict_y = clf.predict(test_x)
    recall = recall_score(test_y,clf.predict(test_x),average = 'macro')
    precision = precision_score(test_y, clf.predict(test_x), average='macro')

    print('accuracy：',clf.score(np.array(test_x), np.array(test_y))) 
    print('recall：',recall)
    print('precision：',precision)
    print('f1_score',2*recall*precision/(recall+precision))

    vote_predict_y = vote(predict_y,datas[-len(test_x):])
    vote_recall = recall_score(test_y,vote_predict_y,average = 'macro')
    vote_precision = precision_score(test_y, vote_predict_y, average='macro')

    print('voting accuracy：',accuracy(vote_predict_y,test_y))
    print('voting recall：',vote_recall)
    print('voting precision：',vote_precision)
    print('voting f1_score',2*vote_recall*vote_precision/(vote_recall+vote_precision))
    print('')

BlVector==============
accuracy： 0.520881510846823
recall： 0.49998793967470384
precision： 0.4271112946520752
f1_score 0.4606853038336461
voting


/home/stocksentiment/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


voting accuracy： 0.5208952758506772
voting recall： 0.5
voting precision： 0.2604476379253386
voting f1_score 0.34249253326092854

DsVector_rate==============
accuracy： 0.5214871710164078
recall： 0.5009265814680646
precision： 0.529247230471489
f1_score 0.5146976225543572
voting
voting accuracy： 0.5209365708622399
voting recall： 0.5001306865504281
voting precision： 0.5153190319154286
voting f1_score 0.507611271218312

SnVector==============
accuracy： 0.5206337407774474
recall： 0.49985037680282735
precision： 0.4860512225962932
f1_score 0.4928542298913098
voting
voting accuracy： 0.5206199757735932
voting recall： 0.4997530309590441
voting precision： 0.41038596104575886
voting f1_score 0.4506819940631131

DsVector==============
accuracy： 0.5242264067833939
recall： 0.5044502805334159
precision： 0.5429033077458991
f1_score 0.522970902968638
voting
voting accuracy： 0.5286036780090299
voting recall： 0.5084375949024605
voting precision： 0.6253113955024281
voting f1_score 0.5608504610545494

PmiVec

In [95]:
# GaussianNB
for vectorname in Xs.keys():
    print(vectorname+'==============')
    X = Xs[vectorname]
    train_x,test_x,train_y,test_y = model_selection.train_test_split(X,Y,test_size=0.2,shuffle=False)

    clf = GaussianNB()

    clf.fit(np.array(train_x), np.array(train_y))

    predict_y = clf.predict(test_x)
    recall = recall_score(test_y,clf.predict(test_x),average = 'macro')
    precision = precision_score(test_y, clf.predict(test_x), average='macro')

    print('accuracy：',clf.score(np.array(test_x), np.array(test_y))) 
    print('recall：',recall)
    print('precision：',precision)
    print('f1_score',2*recall*precision/(recall+precision))

    vote_predict_y = vote(predict_y,datas[-len(test_x):])
    vote_recall = recall_score(test_y,vote_predict_y,average = 'macro')
    vote_precision = precision_score(test_y, vote_predict_y, average='macro')

    print('voting accuracy：',accuracy(vote_predict_y,test_y))
    print('voting recall：',vote_recall)
    print('voting precision：',vote_precision)
    print('voting f1_score',2*vote_recall*vote_precision/(vote_recall+vote_precision))
    print('')

BlVector==============
accuracy： 0.5207851558198436
recall： 0.5010099241239496
precision： 0.5096778012454636
f1_score 0.5053066938877044
voting
voting accuracy： 0.5208677458429688
voting recall： 0.5003469866055271
voting precision： 0.5097657840922931
voting f1_score 0.5050124725558685

DsVector_rate==============
accuracy： 0.48305528025547845
recall： 0.502237350919005
precision： 0.5141714991563476
f1_score 0.5081343627324632
voting
voting accuracy： 0.4815273648276622
voting recall： 0.5014115204290613
voting precision： 0.5148850052055574
voting f1_score 0.5080589508953243

SnVector==============
accuracy： 0.5121269683955512
recall： 0.5002180083164723
precision： 0.5003223382921838
f1_score 0.5002701678648954
voting
voting accuracy： 0.5176880299526484
voting recall： 0.5011960585608762
voting precision： 0.5031678829563104
voting f1_score 0.5021800351597105

DsVector==============
accuracy： 0.48379859046360535
recall： 0.5024287581893708
precision： 0.5118029282265224
f1_score 0.5070725222069

In [25]:
# dnn
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import to_categorical

Y = [np.sign(data['rate']) for data in datas]
for vectorname in Xs.keys():
    print(vectorname+'==============')
    X = Xs[vectorname]
    train_x,test_x,Y1,Y2 = model_selection.train_test_split(X,Y,test_size=0.2,shuffle=False)
    # train_x = [data['DsVector_rate'] for data in datas]
    # Y = [np.sign(data['rate']) for data in datas]
    # test_x = [data['DsVector_rate'] for data in datas2[2000:]]
    # Y2 = [np.sign(data['rate']) for data in datas2[2000:]]

    train_y = []
    for y in Y1:
        if y == 1:
            train_y.append(np.array([0,1]))
        else:    
            train_y.append(np.array([1,0]))

    test_y = []
    for y in Y2:
        if y == 1:
            test_y.append(np.array([0,1]))
        else:    
            test_y.append(np.array([1,0]))

    num_classes = 2
    
#     train_y = to_categorical(train_y,num_classes=num_classes)
#     test_y = to_categorical(test_y,num_classes=num_classes)
    nmodel = Sequential()
    nmodel.add(Dense(units=num_classes, activation = 'relu', input_dim = np.array(train_x).shape[1]))
    nmodel.add(Dropout(0.5))
    nmodel.add(Dense(2, activation = 'relu'))
    nmodel.add(Dropout(0.5))
    # dropout:https://blog.csdn.net/program_developer/article/details/80737724
    nmodel.add(Dense(2, activation = 'softmax'))
    nmodel.compile(loss = 'categorical_crossentropy',
                   optimizer = 'adam',
                   metrics = ['accuracy'])
    nmodel.fit(np.array(train_x),np.array(train_y),epochs=10, batch_size=5)
    print(nmodel.evaluate(np.array(test_x),np.array(test_y), batch_size=5))

BlVector==============
Epoch 1/10
290589/290589 [==============================] - 93s 319us/step - loss: 0.7055 - acc: 0.5070
Epoch 2/10
290589/290589 [==============================] - 93s 322us/step - loss: 0.6931 - acc: 0.5073
Epoch 3/10
290589/290589 [==============================] - 93s 321us/step - loss: 0.6931 - acc: 0.5072
Epoch 4/10
290589/290589 [==============================] - 92s 315us/step - loss: 0.6931 - acc: 0.5071
Epoch 5/10
290589/290589 [==============================] - 92s 316us/step - loss: 0.6931 - acc: 0.5080
Epoch 6/10
290589/290589 [==============================] - 92s 318us/step - loss: 0.6931 - acc: 0.5080
Epoch 7/10
290589/290589 [==============================] - 93s 319us/step - loss: 0.6931 - acc: 0.5074
Epoch 8/10
290589/290589 [==============================] - 93s 319us/step - loss: 0.6931 - acc: 0.5077
Epoch 9/10
290589/290589 [==============================] - 92s 318us/step - loss: 0.6931 - acc: 0.5073
Epoch 10/10
72648/72648 [================

In [40]:
# rnn
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import to_categorical
from keras.layers import LSTM

for vectorname in Xs.keys():
    print(vectorname+'==============')
    X1 = Xs[vectorname]
    X = []
    for idx in tqdm(range(0,len(X1))):
        X.append([[X1[idx],1]])
    X = np.array(X)
    X = np.concatenate(X, axis=0)
    print(X.shape)
    train_x,test_x,Y1,Y2 = model_selection.train_test_split(X,Y,test_size=0.2,shuffle=False)

    train_y = []
    for y in Y1:
        if y == 1:
            train_y.append(np.array([0,1]))
        else:    
            train_y.append(np.array([1,0]))

    test_y = []
    for y in Y2:
        if y == 1:
            test_y.append(np.array([0,1]))
        else:    
            test_y.append(np.array([1,0]))

    num_classes = 2
    neurons = 2                 
    activation_function = 'tanh'  
    loss = 'mse'                  
    optimizer="adam"              
    dropout = 0.25                 
    batch_size = 12               
    epochs = 53
    
    model = Sequential()
    model.add(LSTM(2, input_shape=(train_x.shape[1], train_x.shape[2])))
#     model.add(Dropout(dropout))
#     model.add(LSTM(neurons, return_sequences=True, activation=activation_function))
#     model.add(Dropout(dropout))
#     model.add(LSTM(neurons, activation=activation_function))
#     model.add(Dropout(dropout))
#     model.add(Dense(units=1))
#     model.add(Activation(activation_function))
    model.compile(loss=loss, optimizer=optimizer, metrics=['mae'])
    
    model.fit(np.array(train_x),np.array(train_y),epochs=10, batch_size=5)



100%|██████████| 21805/21805 [00:00<00:00, 409323.55it/s]

BlVector==============
(21805, 2)


IndexError: tuple index out of range

In [107]:
# company Apple
appledatas = [data for data in datas if data['company'] =='Apple Inc.']
apple_count = train_sent_dict(appledatas)
news2vector(appledatas,apple_count,bl_sent)

apple_X1 = [data['DsVector'] for data in appledatas]
apple_X2 = [data['SnVector'] for data in appledatas]
apple_X3 = [data['BlVector'] for data in appledatas]
apple_X4 = [data['PmiVector'] for data in appledatas]
# X = [data['ContextVector'] for data in datas]
apple_X5 = [data['DsVector_rate'] for data in appledatas]

apple_Xs = {'DsVector':apple_X1,'SnVector':apple_X2,'BlVector':apple_X3,'PmiVector':apple_X4,'DsVector_rate':apple_X5}

Y = [np.sign(data['rate']) for data in appledatas]

for vectorname in apple_Xs.keys():
    print(vectorname+'==============')
    X = apple_Xs[vectorname]
    train_x,test_x,train_y,test_y = model_selection.train_test_split(X,Y,test_size=0.2,shuffle=False)

    clf = GaussianNB()
    
    clf.fit(np.array(train_x), np.array(train_y))
    predict_y = clf.predict(test_x)
    recall = recall_score(test_y,clf.predict(test_x),average = 'macro')
    precision = precision_score(test_y, clf.predict(test_x), average='macro')

    print('recall：',recall)
    print('precision：',precision)
    print('f1_score',2*recall*precision/(recall+precision))

    vote_predict_y = vote(predict_y,datas[-len(test_x):])
    vote_recall = recall_score(test_y,vote_predict_y,average = 'macro')
    vote_precision = precision_score(test_y, vote_predict_y, average='macro')

    print('voting accuracy：',accuracy(vote_predict_y,test_y))
    print('voting recall：',vote_recall)
    print('voting precision：',vote_precision)
    print('voting f1_score',2*vote_recall*vote_precision/(vote_recall+vote_precision))
    print('')

















  0%|          | 0/21805 [00:00<?, ?it/s]














 77%|███████▋  | 16837/21805 [00:59<00:21, 230.82it/s]















  1%|          | 258/21805 [00:00<00:08, 2571.88it/s]















  2%|▏         | 516/21805 [00:00<00:08, 2568.24it/s]















  4%|▎         | 776/21805 [00:00<00:08, 2576.59it/s]















  5%|▍         | 991/21805 [00:00<00:08, 2421.26it/s]















  6%|▌         | 1240/21805 [00:00<00:08, 2439.47it/s]















  7%|▋         | 1493/21805 [00:00<00:08, 2465.81it/s]















  8%|▊         | 1774/21805 [00:00<00:07, 2558.61it/s]















  9%|▉         | 2027/21805 [00:00<00:07, 2531.13it/s]















 10%|█         | 2267/21805 [00:00<00:07, 2471.04it/s]















 12%|█▏        | 2510/21805 [00:01<00:07, 2453.61it/s]















 13%|█▎        | 2755/21805 [00:01<00:07, 2450.55it/s]















 14%|█▎        | 2996/21805 [00:01<00:07, 2404.88it/s]















 15%|█▍        | 3234/

BlVector==============
recall： 0.5030578287994746
precision： 0.513431391040007
f1_score 0.5081916773399072
voting
voting accuracy： 0.5166246273790415
voting recall： 0.5032122199489782
voting precision： 0.549419056501127
voting f1_score 0.5253014788551367

DsVector_rate==============
recall： 0.5094570104316638
precision： 0.5233958096404514
f1_score 0.5163323549492113
voting
voting accuracy： 0.4916303600091722
voting recall： 0.5033936584914079
voting precision： 0.5120534259151274
voting f1_score 0.5076866168070631

SnVector==============
recall： 0.5093620814494877
precision： 0.5387957051147861
f1_score 0.523665626208622
voting
voting accuracy： 0.5141022701215318
voting recall： 0.5006688178289679
voting precision： 0.5107294325810068
voting f1_score 0.5056490875619778

DsVector==============
recall： 0.5063655039444992
precision： 0.515294145015723
f1_score 0.5107908092212332
voting
voting accuracy： 0.4916303600091722
voting recall： 0.5031007678512793
voting precision： 0.5097846402767316
vot

In [96]:
# company Apple Randomforest

for vectorname in Xs.keys():
    print(vectorname+'==============')
    X = apple_Xs[vectorname]
    train_x,test_x,train_y,test_y = model_selection.train_test_split(X,Y,test_size=0.2,shuffle=False)

    clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
    
    clf.fit(np.array(train_x), np.array(train_y))
    predict_y = clf.predict(test_x)
    recall = recall_score(test_y,clf.predict(test_x),average = 'macro')
    precision = precision_score(test_y, clf.predict(test_x), average='macro')

    print('recall：',recall)
    print('precision：',precision)
    print('f1_score',2*recall*precision/(recall+precision))

    vote_predict_y = vote(predict_y,datas[-len(test_x):])
    vote_recall = recall_score(test_y,vote_predict_y,average = 'macro')
    vote_precision = precision_score(test_y, vote_predict_y, average='macro')

    print('voting accuracy：',accuracy(vote_predict_y,test_y))
    print('voting recall：',vote_recall)
    print('voting precision：',vote_precision)
    print('voting f1_score',2*vote_recall*vote_precision/(vote_recall+vote_precision))
    print('')

BlVector==============
recall： 0.49998793967470384
precision： 0.4271112946520752
f1_score 0.4606853038336461
voting


/home/stocksentiment/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


voting accuracy： 0.5208952758506772
voting recall： 0.5
voting precision： 0.2604476379253386
voting f1_score 0.34249253326092854

DsVector_rate==============
recall： 0.5009265814680646
precision： 0.529247230471489
f1_score 0.5146976225543572
voting
voting accuracy： 0.5209365708622399
voting recall： 0.5001306865504281
voting precision： 0.5153190319154286
voting f1_score 0.507611271218312

SnVector==============
recall： 0.49985037680282735
precision： 0.4860512225962932
f1_score 0.4928542298913098
voting
voting accuracy： 0.5206199757735932
voting recall： 0.4997530309590441
voting precision： 0.41038596104575886
voting f1_score 0.4506819940631131

DsVector==============
recall： 0.5044502805334159
precision： 0.5429033077458991
f1_score 0.522970902968638
voting
voting accuracy： 0.5286036780090299
voting recall： 0.5084375949024605
voting precision： 0.6253113955024281
voting f1_score 0.5608504610545494

PmiVector==============
recall： 0.5061807470269806
precision： 0.5518193638132911
f1_score 0.5

In [108]:
# company Apple DNN
for vectorname in Xs.keys():
    print(vectorname+'==============')
    X = apple_Xs[vectorname]
    train_x,test_x,Y1,Y2 = model_selection.train_test_split(X,Y,test_size=0.2,shuffle=False)
    # train_x = [data['DsVector_rate'] for data in datas]
    # Y = [np.sign(data['rate']) for data in datas]
    # test_x = [data['DsVector_rate'] for data in datas2[2000:]]
    # Y2 = [np.sign(data['rate']) for data in datas2[2000:]]

    train_y = []
    for y in Y1:
        if y == 1:
            train_y.append(np.array([0,1]))
        else:    
            train_y.append(np.array([1,0]))

    test_y = []
    for y in Y2:
        if y == 1:
            test_y.append(np.array([0,1]))
        else:    
            test_y.append(np.array([1,0]))

    num_classes = 2
    
#     train_y = to_categorical(train_y,num_classes=num_classes)
#     test_y = to_categorical(test_y,num_classes=num_classes)
    nmodel = Sequential()
    nmodel.add(Dense(units=num_classes, activation = 'relu', input_dim = np.array(train_x).shape[1]))
    nmodel.add(Dropout(0.5))
    nmodel.add(Dense(2, activation = 'relu'))
    nmodel.add(Dropout(0.5))
    # dropout:https://blog.csdn.net/program_developer/article/details/80737724
    nmodel.add(Dense(2, activation = 'softmax'))
    nmodel.compile(loss = 'categorical_crossentropy',
                   optimizer = 'adam',
                   metrics = ['accuracy'])
    nmodel.fit(np.array(train_x),np.array(train_y),epochs=10, batch_size=5)
    print(nmodel.evaluate(np.array(test_x),np.array(test_y), batch_size=5))

BlVector==============
Epoch 1/10
17444/17444 [==============================] - 7s 385us/step - loss: 0.7225 - acc: 0.4997
Epoch 2/10
17444/17444 [==============================] - 5s 315us/step - loss: 0.6961 - acc: 0.5000
Epoch 3/10
17444/17444 [==============================] - 6s 342us/step - loss: 0.6941 - acc: 0.5024
Epoch 4/10
17444/17444 [==============================] - 6s 344us/step - loss: 0.6934 - acc: 0.4985
Epoch 5/10
17444/17444 [==============================] - 6s 343us/step - loss: 0.6933 - acc: 0.4981
Epoch 6/10
17444/17444 [==============================] - 6s 349us/step - loss: 0.6933 - acc: 0.4915
Epoch 7/10
17444/17444 [==============================] - 6s 344us/step - loss: 0.6933 - acc: 0.5014
Epoch 8/10
17444/17444 [==============================] - 6s 337us/step - loss: 0.6933 - acc: 0.4981
Epoch 9/10
17444/17444 [==============================] - 6s 350us/step - loss: 0.6933 - acc: 0.5004
Epoch 10/10
4361/4361 [==============================] - 1s 312us/st

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import to_categorical

Y = [np.sign(data['rate']) for data in datas]
for vectorname in apple_Xs.keys():
    print(vectorname+'==============')
    X = apple_Xs[vectorname]
    train_x,test_x,Y1,Y2 = model_selection.train_test_split(X,Y,test_size=0.2,shuffle=False)
    # train_x = [data['DsVector_rate'] for data in datas]
    # Y = [np.sign(data['rate']) for data in datas]
    # test_x = [data['DsVector_rate'] for data in datas2[2000:]]
    # Y2 = [np.sign(data['rate']) for data in datas2[2000:]]

    train_y = []
    for y in Y1:
        if y == 1:
            train_y.append(np.array([0,1]))
        else:    
            train_y.append(np.array([1,0]))

    test_y = []
    for y in Y2:
        if y == 1:
            test_y.append(np.array([0,1]))
        else:    
            test_y.append(np.array([1,0]))

    num_classes = 2
    model = Sequential()
    model.add(LSTM(neurons, return_sequences=True, 
    input_shape=(inputs.shape[1], inputs.shape[2]), activation=activ_func))
    model.add(Dropout(dropout))
    model.add(LSTM(neurons, return_sequences=True, activation=activ_func))
    model.add(Dropout(dropout))
    model.add(LSTM(neurons, activation=activ_func))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))
    model.compile(loss=loss, optimizer=optimizer, metrics=['mae'])
    model.summary()

In [191]:
## iterate
dates = [data['date'] for data in datas]
idxs = []
for date in ['2016-12-29','2017-01-30','2017-02-28','2017-03-30','2017-04-28','2017-05-30','2017-06-30','2017-07-31','2017-08-30','2017-09-29','2017-10-30','2017-11-30','2017-12-29']:
    idxs.append(dates.index(date))

In [192]:
idxs

[318950,
 321630,
 324727,
 327362,
 330253,
 333254,
 336362,
 339416,
 342353,
 345361,
 348815,
 352002,
 354236]

In [354]:
new_counts = []
new_datas_list = []
date_list = ['2016-12-29','2017-01-30','2017-02-28','2017-03-30','2017-04-28','2017-05-30','2017-06-30','2017-07-31','2017-08-30','2017-09-29','2017-10-30','2017-11-30','2017-12-29']
for i in range(0,len(idxs)-1):
    new_count = train_sent_dict(datas[:idxs[i]])
    new_datas = datas.copy()
    news2vector2(new_datas,new_count,bl_sent)
    print(date_list[i])
    X = [data['DsVector_rate'] for data in new_datas]
    Y = [np.sign(data['rate']) for data in new_datas]
    train_x = X[0:idxs[i]]
    train_y = Y[0:idxs[i]]
    test_x = X[idxs[i]:idxs[i+1]] 
    test_y = Y[idxs[i]:idxs[i+1]]

    clf.fit(np.array(train_x), np.array(train_y))
    predict_y = clf.predict(test_x)
    recall = recall_score(test_y,clf.predict(test_x),average = 'binary')
    precision = precision_score(test_y, clf.predict(test_x), average='binary')
    clf = GaussianNB()

#     print('recall：',recall)
#     print('precision：',precision)
#     print('f1_score',2*recall*precision/(recall+precision))

    vote_predict_y = vote(predict_y,datas[-len(test_x):])
    vote_recall = recall_score(test_y,vote_predict_y,average = 'binary')
    vote_precision = precision_score(test_y, vote_predict_y, average='binary')

    print('voting accuracy：',accuracy(vote_predict_y,test_y))
    print('voting recall：',vote_recall)
    print('voting precision：',vote_precision)
    print('voting f1_score',2*vote_recall*vote_precision/(vote_recall+vote_precision))
    print('')
    print('===========')

2016-12-29
voting
voting accuracy： 0.5055970149253731
voting recall： 0.01664145234493192
voting precision： 0.46808510638297873
voting f1_score 0.03214024835646457

2017-01-30
voting
voting accuracy： 0.4304165321278657
voting recall： 0.038396386222473176
voting precision： 0.5271317829457365
voting f1_score 0.07157894736842105

2017-02-28
voting
voting accuracy： 0.4990512333965844
voting recall： 0.027169811320754716
voting precision： 0.5373134328358209
voting f1_score 0.05172413793103449

2017-03-30
voting
voting accuracy： 0.5264614320304393
voting recall： 0.019103600293901544
voting precision： 0.43333333333333335
voting f1_score 0.036593947923997186

2017-04-28
voting
voting accuracy： 0.4828390536487837
voting recall： 0.02192134107027724
voting precision： 0.4927536231884058
voting f1_score 0.04197530864197531

2017-05-30
voting
voting accuracy： 0.4874517374517375
voting recall： 0.029746835443037974
voting precision： 0.4392523364485981
voting f1_score 0.05572021339656194

2017-06-30
voti

In [332]:
X = [data['DsVector_rate'] for data in new_datas]

In [339]:
datas[2121]['DsVector_rate']

[0.17790151913386223,
 0.01895255343326669,
 0.47347614740668953,
 0.2625919648718393]

In [331]:
news2vector(datas,count,bl_sent)